In [ ]:
# --- Install ipywidgets for sliders ---
!pip install ipywidgets --quiet

import cv2, numpy as np, matplotlib.pyplot as plt
from google.colab import files
from ipywidgets import interact, IntSlider, Dropdown, Checkbox
from IPython.display import display
from PIL import Image

# --- Step 1: Upload function ---
def upload_image():
    uploaded = files.upload()
    fname = list(uploaded.keys())[0]
    img = cv2.imdecode(np.frombuffer(uploaded[fname], np.uint8), cv2.IMREAD_COLOR)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Upload once
print("📂 Upload an image (jpg/png):")
img_rgb = upload_image()
last_result = None

# --- Step 2: Artistic Filters ---
def apply_filter(d=9, sigmaColor=100, sigmaSpace=100,
                 median_k=7, edge_block=9, edge_C=2,
                 preset="None", save=False, view=False):
    global last_result, img_rgb

    # 1. Smooth colors (bilateral filter with better ranges)
    color = cv2.bilateralFilter(img_rgb, d, sigmaColor, sigmaSpace)

    # 2. Edge detection
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    gray_blur = cv2.medianBlur(gray, median_k)
    edges = cv2.adaptiveThreshold(
        gray_blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        blockSize=edge_block, C=edge_C
    )

    # 3. Combine cartoon
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    # --- Presets ---
    if preset == "Cartoon1":
        cartoon = cv2.bilateralFilter(cartoon, 12, 200, 200)
    elif preset == "ComicBook":
        edges_inv = cv2.bitwise_not(edges)
        cartoon = cv2.addWeighted(color, 0.6, cv2.cvtColor(edges_inv, cv2.COLOR_GRAY2RGB), 0.4, 0)
    elif preset == "SoftPaint":
        cartoon = cv2.edgePreservingFilter(img_rgb, flags=1, sigma_s=100, sigma_r=0.4)
    elif preset == "PencilSketch":
        sketch_gray, sketch_color = cv2.pencilSketch(img_rgb, sigma_s=60, sigma_r=0.07, shade_factor=0.05)
        cartoon = sketch_color
    elif preset == "PopArt":
        hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        hsv[:,:,1] = cv2.equalizeHist(hsv[:,:,1])
        cartoon = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # Save last result
    last_result = cartoon

    # --- Display Bigger + Styling ---
    plt.figure(figsize=(24,10))
    plt.subplot(1,3,1); plt.imshow(img_rgb); plt.title("Original", fontsize=14, weight="bold"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(color); plt.title("Smoothed Colors", fontsize=14, weight="bold"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(cartoon); plt.title(f"Artistic Filter: {preset}", fontsize=14, weight="bold"); plt.axis("off")
    plt.show()

    # --- Save option ---
    if save:
        out_name = "artistic_output.png"
        cv2.imwrite(out_name, cv2.cvtColor(cartoon, cv2.COLOR_RGB2BGR))
        files.download(out_name)

    # --- Full View option ---
    if view:
        display(Image.fromarray(cartoon))

# --- Step 3: Interactive UI ---
interact(
    apply_filter,
    d=IntSlider(9, min=3, max=25, step=2, description="Smooth Size"),
    sigmaColor=IntSlider(100, min=50, max=300, step=20, description="Sigma Color"),
    sigmaSpace=IntSlider(100, min=50, max=300, step=20, description="Sigma Space"),
    median_k=IntSlider(7, min=3, max=15, step=2, description="Median Blur"),
    edge_block=IntSlider(9, min=3, max=21, step=2, description="Edge Block Size"),
    edge_C=IntSlider(2, min=0, max=10, step=1, description="Edge Threshold"),
    preset=Dropdown(options=["None","Cartoon1","ComicBook","SoftPaint","PencilSketch","PopArt"],
                    value="None", description="Preset"),
    save=Checkbox(value=False, description="💾 Save Image"),
    view=Checkbox(value=False, description="🔍 Full View")
)

print("📌 Tip: Run this cell again if you want to re-upload a new image.")


📂 Upload an image (jpg/png):
